In [1]:
# 第一题
from multiprocessing.sharedctypes import Value
from turtle import speed
import matplotlib.pyplot as plt  #画图用
import numpy as np
import matplotlib
import xlrd 
import math
import xlwt
cnc=np.empty([6,8],dtype=int) 
# %cnc: 
# % 第一行：cnc位置常量
# % 第二行：cnc工作变量，空闲为0，忙碌为1
# % 第三行：cnc工作时间，为常量560
# % 第四行：cnc上下料时间
# % 第五行：cnc预计完成时间
# % 第六行：cnc加工物料数
file1="D:\\数学建模\\practice\\1_cnc_1.xls"
data = xlrd.open_workbook(file1) 
sheet = data.sheet_by_index(0)  #sheet为第一个
# for col in range(sheet.ncols):   
#     print(sheet.cell_value(0,col)) 
for row in range(sheet.nrows):
    for col in range(sheet.ncols):
        cnc[row,col]=sheet.cell_value(row,col)
cnc[5,:]=-1
#print(cnc)
# %rgv：
# % 第一行：rgv初始位置变量，初始为1
# % 第二行：rgv工作变量，进行上下料或清洗作业，空闲为0，忙碌为1
# % 第三行：rgv一次清洗作业时间
# % 第四行：rgv预计完成时间
rgv=np.empty([4,1],dtype=int)
file2="D:\\数学建模\\practice\\1_rgv_1.xls"
data=xlrd.open_workbook(file2)
sheet=data.sheet_by_index(0)
for row in range(sheet.nrows):
    rgv[row,0]=sheet.cell_value(row,0)
#print(rgv)
# %speedRgv：
# %移动1~3个单位所需时间

speedRgv=np.empty([3,1],dtype=int)
file3="D:\\数学建模\\practice\\1_speedRgv_1.xls"
data= xlrd.open_workbook(file3)
sheet= data.sheet_by_index(0)
for row in range(sheet.nrows):
    speedRgv[row,0]=sheet.cell_value(row,0)
#print(speedRgv)
case_1_result = np.zeros([800,2])
t,r=0,0#计数
while t<8*60*60:
    cncNum=0#cnc的编号
    value=0 #cnc的位置
    step=0#rgv走的步数
    #%1.在rgv和cnc均空闲状态下，找到距离最近的cnc编号
    dist=4
    signal=0
    for i in range(0,8):
        if cnc[1,i]==0:
            if abs(cnc[0,i]-rgv[0,0])<dist:
                dist=abs(cnc[0,i]-rgv[0,0])
                cncNum=i
                value=cnc[0,i]
                step=abs(cnc[0,i]-rgv[0,0]) 
    if step!=0 and cnc[1,cncNum]==0:
        t=t+speedRgv[step-1,0]
        rgv[0,0]=value
        signal=1
    if cnc[1,cncNum]==0 :
        cnc[1,cncNum]=1
        rgv[1,0]=1
        #记录加工cnc的编号
        case_1_result[r,0] = cncNum
        case_1_result[r,1] = t
        t = t + cnc[3,cncNum]; 
        cnc[5,cncNum] +=1
        signal = 1
        cnc[4,cncNum] = t + cnc[2,cncNum];  #%cnc预计完成时间=上下料+工作时间
        if cnc[4,cncNum]!=0:
            t=t+rgv[2,0]
        r=r+1
    for i in range(0,8):
        if t >=cnc[4,i] and cnc[4,i]!=0:
            cnc[1,i]=0
    if signal==0:
        t=t+1

file4 = xlwt.Workbook()
sheet= file4.add_sheet("sheet1",cell_overwrite_ok=True)
sheet.write(0,0,"cnc编号")
sheet.write(0,1,"cnc时间")
sheet.write(0,2,"cnc下料开始时间/上料时间")

for i in range(0,800):
    for j in range(0,2):
        sheet.write(i+1,j,case_1_result[i,j])

for i in range(0,800):
    for j in range(i+1,800):
        if case_1_result[i,0]==case_1_result[j,0]:
            sheet.write(i+1,2,case_1_result[j,1])
            break

file4.save("1_case_1_result-1.xls")











In [3]:
from imp import load_compiled
from turtle import distance
from unittest import result
import xlrd 
import numpy as np
import xlwt

# %1.cnc: 
# % 第一行：cnc位置常量
# % 第二行：cnc工作变量，空闲为0，忙碌为1
# % 第三行：cnc工作时间
# % 第四行：cnc上下料时间
# % 第五行：上料第几次
# % 第六行：正在加工的物料编号
# % 第七行：cnc是否初始判断，0为初始，1位正常

cnc=np.zeros([7,8])

# %2.rgv：
# % 第一行：rgv初始位置变量，初始为1
# % 第二行：rgv工作变量，进行上下料或清洗作业，空闲为0，忙碌为1
# % 第三行：rgv一次清洗作业时间
# % 第四行：rgv预计完成时间
# % 第五行：正在操作的物料编号
rgv=np.zeros([5,1])
t=0
num1=0
num2=0
lis1=[6,4,2,0]
lis2=[7,5,3,1]
flag=1
r=0#编号
n=1 #计数
n1=1#记时
result_cnc=np.zeros([3,800])
speedRgv=np.array([[20],[33],[46]])

file1="D:\\数学建模\\practice\\2_cnc_1.xls"
data=xlrd.open_workbook(file1)
sheet=data.sheet_by_index(0)
for row in range(sheet.nrows)  :
    for col in range(sheet.ncols) :
        cnc[row,col]=sheet.cell_value(row,col)
#print(cnc)
file2="D:\\数学建模\\practice\\2_rgv_1.xls"
data=xlrd.open_workbook(file2)
sheet=data.sheet_by_index(0)
for row in range(sheet.nrows):
    for col in range(sheet.ncols):
        rgv[row,col]=sheet.cell_value(row,col)
lt=2
#print(rgv)
#


while t<8*3600:
    cncNum1=-1
    cncNum2=-1
    loco=-1
    step=0
    dist1=4
    dist2=4
#找距离最近的cnc
    for i in lis1:
        if cnc[1,i]!=1:
            if abs(cnc[0,i]-rgv[0,0])<dist1:
                dist1=abs(cnc[0,i]-rgv[0,0])
                cncNum1=i
                loco=cnc[0,i]
                result_cnc[1,n]=cncNum1
                n+=1
                step=abs(cnc[0,cncNum1]-rgv[0,0])
                
#rgv移动
    if rgv[1,0]!=1 :
        rgv[0,0]=loco
        t=t+speedRgv[int(step),0]
#rgv进行上下料
    if cnc[1,cncNum1]!=1:
        num1+1
        rgv[2,0]=1
        t=t+cnc[3,cncNum1]
        result_cnc[2,n1]=t
        n1+=1
        r+=1
        rgv[4]=r
        cnc[5,cncNum1]=r
        rgv[2,0]==0
        cnc[6,cncNum1]=1
        cnc[4,cncNum1]+=1
        if cnc[6,cncNum1]!=0:
            flag=1
            
#第二步取cnc
    if rgv[1,0]!=1 and flag==1: 
        for i in lis2:
            if cnc[1,i]!=1:
                if abs(cnc[0,i]-rgv[0,0])<dist2:
                    dist2=abs(cnc[0,i]-rgv[0,0])
                    cncNum2=i
                    loco=cnc[0,i]
                    result_cnc[1,n]=cncNum2
                    n+=1
                    step=abs(cnc[0,cncNum2]-rgv[0,0])
                    
    if rgv[1,0]!=1 :
        rgv[0,0]=loco
        t=t+speedRgv[int(step),0]
#第二步上料
    if cnc[1,cncNum2]!=1 and flag==1:
        num2+1
        rgv[2,0]=1
        t=t+cnc[3,cncNum2]
        result_cnc[2,n1]=t
        n1+=1
        r+=1
        rgv[4]=r
        cnc[5,cncNum2]=r
        rgv[2,0]==0
        cnc[6,cncNum2]=1
        cnc[4,cncNum2]+=1
        flag=0
            
#判断工作完成时间
    for i in range(0,8):
        if t>(cnc[4,i]*cnc[2,i]):
            cnc[1,i]=0
        if t>(cnc[4,i]*cnc[2,i]):
            cnc[1,i]=0
            t=t+rgv[2,0]
#cnc都在忙碌
    k=-1
    for i in range(0,8):
        min=1000000
        
        if cnc[5,i]< min:
            min=cnc[5,i]
            k=i
    t=t+cnc[2,i]




x=a = np.transpose(result_cnc)

file4 = xlwt.Workbook()
sheet= file4.add_sheet("sheet1",cell_overwrite_ok=True)

for i in range(0,3):
    for j in range(0,800):
        sheet.write(j,i,result_cnc[i,j])

file4.save("result_cnc3.xls")







In [20]:

x=8*3600

print(x)


28800
